### Forwards optimization notebook

By forward optimization here I mean optimization that only tunes existing parameters.
AKA finding the optimal value for any given parameter the strategy has so that it yields better results.

This notebook aims to create an interface json which can later be easily read for the optimal parameters for a given timeframe (start to date and period before to date).

for start_date in range(first_date, last_date):
    for end_date in range(start_date, last_date):
        generate_list of stuff
        
for thing in generated list not yet parsed:
    run the computation cronjob
    save the parsed config for later
    
def compute_optimal_stuff():
    filter the dataset & save it to disk maybe?
    groub by `each parameter` and apply function
    save the results for each parameter - function combination list
    
def save_otpimal_stuff()
    open the json
    save under date range key
    
results_json = {
    'date currently at': {
        '1-month': {
            'mean': {
                'sma': ..,
                'rsi close': ..
                ...
            },
            'order': ..
            'linear': ..
        }
        '3-month': ..
        '6-month': ..
        '12-month': ..
        'max': ..
    }
}

If a date range is empty, the next one in line will be selected. If all are empty, it will use the default params.
It does NOT account for how many trades there are out there?


### Imports and variables

In [1]:
import datetime 
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule

import cudf
import dask_cudf

import pandas as pd
import tqdm
# from matplotlib import pyplot
# import seaborn as sns
import dask.dataframe as dd

TIMEFRAME = 'daily'
# TIMEFRAME = 'weekly'

FRAMEWORK = 'dask'
# FRAMEWORK = 'pandas'

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/forward_opt_results.json'

### Utils

In [2]:
def get_sample(the_df):
    print(f'There are {len(the_df)} rows in the dataframe/series')
    try:
        res = the_df.sample(frac=5)
    except ValueError:
        res = the_df.sample(3)
    
    return res

In [3]:
def save_results(key, results_for_key):
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
        
    results_dict[key] = results_for_key
    
    with open(FORWARD_OPT_RESULTS_JSON, 'w') as f:
        f.write(json.dumps(results_dict))

# save_results('a', {1:2,3:4})

In [4]:
def get_pair_entry(start_date, end_date):
    return (
        f"{start_date.strftime('%y-%m')}-to-{end_date.strftime('%y-%m')}",
        start_date,
        end_date
    )

def generate_date_ranges():
    resulting_dict = {}
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE):
        resulting_dict[end_date.strftime('%y-%m')] = {
#             '1-month': get_pair_entry(end_date + relativedelta(months=-1), end_date),
#             '3-month': get_pair_entry(end_date + relativedelta(months=-3), end_date),
            '6-month': get_pair_entry(end_date + relativedelta(months=-6), end_date),
            '12-month': get_pair_entry(end_date + relativedelta(months=-12), end_date),
            'max': get_pair_entry(datetime.datetime(1998, 1, 1), end_date)
        }

    return resulting_dict


def get_parsed_date_ranges():
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
    
    return list(results_dict.keys())

# for k,v in generate_date_ranges().items():
#     print(k)
#     print(v)
# #     range_name, key, start_date, end_date = v
# #     print(range_name, key, start_date, end_date)
#     raise Exception

### Load the dataframe

In [5]:
if TIMEFRAME == 'daily':
    path = 'dataset/daily/'
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx8",
        "adx16",
        "adx32",
        "ppo8",
        "ppo16",
        "ppo32",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]
elif TIMEFRAME == 'weekly':
    path = 'dataset/weekly/'    
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx3",
        "adx6",
        "adx9",
        "ppo3",
        "ppo6",
        "ppo9",
        "stochastic3",
        "stochastic6",
        "stochastic9",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]

if FRAMEWORK == 'pandas':
    d = pd
    d = cudf
    f = f"{path}/all_results.csv"
elif FRAMEWORK == 'dask':
#     d = dd
    d = dask_cudf
    f = f"{path}/*.csv"

# Read CSV file and assign the right column names
# Comment the first two rows if you don't want to use the entire dataset, and instead just work on a sample
df = d.read_csv(
    f,
    names=column_names,
)

In [6]:
df

,uid,ticker,date,price_open,maperiod,rsi_open_period,adx8,adx16,adx32,ppo8,ppo16,ppo32,stochastic8,stochastic16,stochastic32,price_sell,days_ago_close_period,rsi_close_period
npartitions=10,,,,,,,,,,,,,,,,,,
,object,object,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
df.shape[0].compute()

9726309

In [8]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf

cluster = LocalCUDACluster() # by default use all GPUs in the node. I have two.
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:37345 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 15.48 GiB


In [32]:
!pip install dask-cuda

     |████████████████████████████████| 71 kB 37 kB/s  eta 0:00:01


### Compute the change percentage
And drop the price_sell and price_open columns because they are no longer used

In [10]:
# Add the percentage change for the price
df["perc_change"] = (df["price_sell"] - df["price_open"]) / df["price_open"] * 100

In [11]:
# The prices can now be dropped since I won't need them anymore
df = df.drop(["price_sell", "price_open"], axis=1)

In [12]:
get_sample(df)

There are 9726309 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,ppo8,ppo16,ppo32,stochastic8,stochastic16,stochastic32,days_ago_close_period,rsi_close_period,perc_change
npartitions=10,,,,,,,,,,,,,,,,,
,object,object,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Strategy explanation
## Initial strategy parameters
### SMA
The simple moving average of close prices for the close price of the previous day to be above so that we have the first buy signal
### RSI open
The relative strength index lower barrier to break for a buy signal
### RSI close
The relative strength index barrier that, if it breaks above, indicates a sell signal
### Days to wait before closing a trade
If the RSI close barrier is not reached, how long to hold the trade before selling.

## Forward optimizations goals 
Find the best parameter combination for this strategy by exploring relatively close range of values for each of the previously mentioned parameters.
It's called 'forward' because we already have a strategy defined but we just tweak it's parameters in the hopes of improving results.
This will be explored next, and the focus on 'backward' optimization will fall into a later part of the notebook.


##### It is important to note how the dataset was generated, only the RSI open and SMA indicators were considered and 20 days were recorded.

### Optimization utilities
The idea is simple:

    1. Groub py a field
    2. Find the best value for 'perc_change'
    3. Return the best value for the field that was grouped by
    
The problem is step 2. How do you compute the best perc_change value?
Ideas:

    * The simplest approach is to use the mean. It's easy to compute but doesn't show us the real picture because it doesn't account for the order of trades                     

    * Another option may be to apply these trades in order to one unit. This doesn't account for random starting times.                                  
    
    * Another option may be to apply these trader in order to one unit, multiple times, each time dropping one entry from beggining to end. Finally, find the mean of these resulting values. The advantage is that it accounts for order but also for random starting points.                       

In [161]:
def mean_optimal_func(tmp_df_groupby):
    tmp_series = tmp_df_groupby["perc_change"].mean()

    if FRAMEWORK == 'dask':
        tmp_series = tmp_series.compute()
    
    return tmp_series.sort_values(ascending=False).index[0]


def order_optimal_func(tmp_df_groupby):
    def outcome_computation(x):
        l = []
        print('-asd----')
        print(x)
        print(x.sort_values(['date']))
        print('asdasdasd')
        for _, perc_change in x.sort_values(['date'])['perc_change'].iteritems():
            l.append(perc_change)

        results = []
        for i in range(len(l)):
            res = 1
            for j in range(i, len(l)):
                res += (res * l[j]) / 100
            results.append(res)

        return sum(results) / len(results)

    if FRAMEWORK == 'pandas':
        outcome_series = tmp_df_groupby.apply(outcome_computation)
        print('aaa')
        res = outcome_series.sort_values(ascending=False).index[0]
    else:
        outcome_series = tmp_df_groupby.apply(outcome_computation, meta=('float'))
        print(outcome_series)
        print('-----')
        print(outcome_series.compute())
        res = outcome_series.compute().sort_values(ascending=False).index[0]
    
    return res


def linear_optimal_func(df):
    pass


def compute_optimal_params_using_func(df, func):
    optimal_days_ago = int(func(df.groupby(['days_ago_close_period'])))
    df = df[df['days_ago_close_period'] == optimal_days_ago]
    
    optimal_rsi_open = int(func(df.groupby(['rsi_open_period'])))
    df = df[df['rsi_open_period'] == optimal_rsi_open]
    
    optimal_maperiod = int(func(df.groupby(['maperiod'])))
    df = df[df['maperiod'] == optimal_maperiod]
    
    result = {
        'days_ago_close_period': optimal_days_ago,
        'rsi_open_period': optimal_rsi_open,
        'maperiod': optimal_maperiod
    }
    
    return result 


def compute_optimal_results(df):
    return {
        'mean': compute_optimal_params_using_func(df, mean_optimal_func),
        'order': compute_optimal_params_using_func(df, order_optimal_func),
#         'linear': compute_optimal_params_using_func(filtered_df, linear_optimal_func),
    }


In [25]:
def outcome_computation(x):
    l = []

    for perc_change in x.sort_values(['date'])['perc_change'].values:
        l.append(perc_change)

    results = []
    for i in range(len(l)):
        res = 1
        for j in range(i, len(l)):
            res += (res * l[j]) / 100
        results.append(res)

    return sum(results) / len(results)

a = my_df.groupby(['days_ago_close_period']).apply(outcome_computation, meta=('float'))
a

Dask Series Structure:
npartitions=10
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 300 tasks

In [29]:
%%time
a.compute()

KeyboardInterrupt: 

In [28]:
%%time
my_df.shape[0].compute()

CPU times: user 185 ms, sys: 4.9 ms, total: 190 ms
Wall time: 4.93 s


80430

In [168]:
my_df2 = my_df.reset_index(drop=True)

In [14]:
def filter_df_by_dates(df, start_date, end_date):
    return df[(df['date'] >= start_date.strftime('%Y-%m-%d')) & (df['date'] <= end_date.strftime('%Y-%m-%d'))]

In [151]:
STRATEGY_OUT_OF_THE_BOX_KWARGS = {
    'maperiod': 200,
    'rsi_open_period': 10,
    'days_ago_close_period': 10
} if TIMEFRAME == 'daily' else {
    'maperiod': 40,
    'rsi_open_period': 2,
    'days_ago_close_period': 2
}
DEFAULT_STRATEGY_KWARGS = {
    'mean': STRATEGY_OUT_OF_THE_BOX_KWARGS,
    'order': STRATEGY_OUT_OF_THE_BOX_KWARGS,
#     'linear': STRATEGY_OUT_OF_THE_BOX_KWARGS
}

In [152]:
all_date_ranges = generate_date_ranges()
parsed_date_ranges = get_parsed_date_ranges()
unparsed_date_ranges = {k:v for k, v in all_date_ranges.items() if k not in parsed_date_ranges}
# TODO: check 01-11 adn 02-12 (or up to '03-02'?) for daily timeframe
list(unparsed_date_ranges.items())[0]

('03-02',
 {'6-month': ('02-08-to-03-02',
   datetime.datetime(2002, 8, 1, 0, 0),
   datetime.datetime(2003, 2, 1, 0, 0)),
  '12-month': ('02-02-to-03-02',
   datetime.datetime(2002, 2, 1, 0, 0),
   datetime.datetime(2003, 2, 1, 0, 0)),
  'max': ('98-01-to-03-02',
   datetime.datetime(1998, 1, 1, 0, 0),
   datetime.datetime(2003, 2, 1, 0, 0))})

In [153]:
with tqdm.tqdm(total=len(unparsed_date_ranges)) as pbar:

    # Simple caching mechanism to speed things up for a lot of computations
    PREVIOUS_SIZE = None
    PREVIOUS_RES = None
    
    for date_value, date_ranges in unparsed_date_ranges.items():
        optimal_results = {}

        def optimize_range(df, range_name, default_if_empty):
            range_values = date_ranges[range_name]
            _, start_date, end_date = range_values
            
            filtered_df = filter_df_by_dates(df, start_date, end_date)
            size = filtered_df.shape[0] if FRAMEWORK == 'pandas' else filtered_df.shape[0].compute()

            if size == globals()['PREVIOUS_SIZE']:
                print(f'Using chache for {range_name} - {start_date} to {end_date}')
                return globals()['PREVIOUS_RES']
            
            # TODO: Does this account for statistical significant results?
            if size < 100:
                print(f'Empty or too small df from (size: {size}) {start_date} to {end_date}')
                res = default_if_empty
            else:
                print(f'Parsing {size} lines from {start_date} to {end_date}')
                res = compute_optimal_results(filtered_df)

            globals()['PREVIOUS_SIZE'] = size
            globals()['PREVIOUS_RES'] = res
            
            return res
                
        optimal_results['max'] = optimize_range(df, 'max', DEFAULT_STRATEGY_KWARGS)
        optimal_results['12-month'] = optimize_range(df, '12-month', optimal_results['max'])
        optimal_results['6-month'] = optimize_range(df, '6-month', optimal_results['12-month'])
#         optimal_results['3-month'] = optimize_range(df, '3-month', optimal_results['6-month'])
#         optimal_results['1-month'] = optimize_range(df, '1-month', optimal_results['3-month'])

        print(date_value)
        print(optimal_results)
        save_results(date_value, optimal_results)
        pbar.update()
    

  0%|          | 0/215 [00:00<?, ?it/s]

Parsing 1569267 lines from 1998-01-01 00:00:00 to 2003-02-01 00:00:00


  0%|          | 0/215 [00:26<?, ?it/s]distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/home/narboom23/anaconda3/envs/licenta/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/home/narboom23/anaconda3/envs/licenta/lib/python3.8/site-packages/distributed/scheduler.py", line 3647, in heartbeat_worker
    ws._executing = {
  File "/home/narboom23/anaconda3/envs/licenta/lib/python3.8/site-packages/distributed/scheduler.py", line 3648, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: "('simple-shuffle-outcome_computation-63359a029f6b95ce59ab213a3b6a01c6', 1)"



TypeError: Series object is not iterable. Consider using `.to_arrow()`, `.to_pandas()` or `.values_host` if you wish to iterate over the values.

In [91]:
my_df = filter_df_by_dates(df, datetime.datetime(2002, 8, 1, 0, 0), datetime.datetime(2003, 2, 1, 0, 0))

In [93]:
my_df.shape[0].compute()

80430

In [94]:
int(mean_optimal_func(my_df.groupby(['days_ago_close_period'])))

0

In [107]:
my_df.groupby(['days_ago_close_period'])['perc_change'].mean().compute()

days_ago_close_period
0     0.127434
1    -0.066450
2    -0.290557
3    -0.405931
4    -0.276692
5    -0.397845
6    -0.422405
7    -0.451459
8    -0.453364
9    -0.606738
10   -0.771582
11   -1.225296
12   -1.313848
13   -1.420777
14   -1.714828
15   -1.888697
16   -1.875106
17   -1.862835
18   -1.857223
19   -1.528334
20   -1.723540
Name: perc_change, dtype: float64

In [108]:
my_df.groupby(['days_ago_close_period'])['perc_change'].mean().idxmax().compute()

0

In [114]:
percdf = my_df.groupby(['days_ago_close_period'])['perc_change'].mean().compute()

In [120]:
percdf.sort_values(ascending=False).iloc[0]

0.1274336636966368

In [125]:
percdf.sort_values(ascending=False).index[0]

0

In [123]:
percdf

days_ago_close_period
0     0.127434
1    -0.066450
2    -0.290557
3    -0.405931
4    -0.276692
5    -0.397845
6    -0.422405
7    -0.451459
8    -0.453364
9    -0.606738
10   -0.771582
11   -1.225296
12   -1.313848
13   -1.420777
14   -1.714828
15   -1.888697
16   -1.875106
17   -1.862835
18   -1.857223
19   -1.528334
20   -1.723540
Name: perc_change, dtype: float64

In [ ]:
==================

In [156]:
order_optimal_func(my_df.groupby(['days_ago_close_period']))

TypeError: Series object is not iterable. Consider using `.to_arrow()`, `.to_pandas()` or `.values_host` if you wish to iterate over the values.

In [ ]:
====================

In [16]:
my_df = filter_df_by_dates(df, datetime.datetime(2002, 8, 1, 0, 0), datetime.datetime(2003, 2, 1, 0, 0))
my_df

,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,ppo8,ppo16,ppo32,stochastic8,stochastic16,stochastic32,days_ago_close_period,rsi_close_period,perc_change
npartitions=10,,,,,,,,,,,,,,,,,
,object,object,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [142]:
my_df.shape[0].compute()

80430

In [143]:
int(mean_optimal_func(my_df.groupby(['days_ago_close_period'])))

AttributeError: 'Series' object has no attribute 'idxmax'

In [15]:
my_df.groupby(['days_ago_close_period'])['perc_change'].mean().compute()

NameError: name 'my_df' is not defined

In [145]:
percdf = my_df.groupby(['days_ago_close_period'])['perc_change'].mean().compute()

In [146]:
percdf.sort_values(ascending=False).iloc[0]

0.1274336636966369

In [147]:
percdf.sort_values(ascending=False).index[0]

0

In [148]:
percdf

days_ago_close_period
13   -1.420777
18   -1.857223
11   -1.225296
2    -0.290557
8    -0.453364
17   -1.862835
14   -1.714828
15   -1.888697
3    -0.405931
0     0.127434
6    -0.422405
7    -0.451459
1    -0.066450
5    -0.397845
16   -1.875106
20   -1.723540
19   -1.528334
4    -0.276692
9    -0.606738
10   -0.771582
12   -1.313848
Name: perc_change, dtype: float64